#   뉴스 분석 대시보드 (Gradio)

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

import re
import json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")


`(3) 뉴스 데이터 로드`

In [ ]:
# pickle 파일에서 데이터 로드
import pickle

with open("processed_news_articles.pkl", "rb") as f:
    loaded_docs = pickle.load(f)
    print(f"로드된 문서 수: {len(loaded_docs)}")

In [ ]:
loaded_docs[0].metadata

---

## **Gradio 대시보드 구조 만들기**


In [ ]:
import gradio as gr

def process_query(query, num_articles):
    """사용자 질문을 처리하고 결과를 반환하는 함수"""

    # 쿼리 전처리
    query = query.strip()

    # 뉴스 기사 검색

    # 요약 
    summary_output = ""

    # 키워드 추출
    keywords_output = []

    # 차트 출력
    chart_output = None

    # 기사별 분석 결과
    articles_output = ""

    # JSON 데이터
    json_output = {}
    
    return summary_output, keywords_output, chart_output, articles_output, json_output


# Gradio 인터페이스 구현
with gr.Blocks(title="뉴스 분석 대시보드", theme=gr.themes.Soft(), analytics_enabled=False) as demo:
    gr.Markdown("# 🔍 뉴스 분석 대시보드")
    gr.Markdown("주제를 입력하면 관련 뉴스를 검색하고 요약, 키워드 추출, 감성 분석을 수행합니다.")
    
    with gr.Row():
        with gr.Column(scale=4):
            query_input = gr.Textbox(label="분석할 주제 입력", placeholder="예: 인공지능, 기후변화, 경제 등")
        with gr.Column(scale=1):
            num_articles = gr.Slider(label="분석할 기사 수", minimum=1, maximum=10, value=3, step=1)
    
    analyze_btn = gr.Button("뉴스 분석하기", variant="primary")
    
    # 결과 출력 영역
    with gr.Tabs():
        with gr.TabItem("요약 및 키워드"):
            summary_output = gr.Textbox(label="분석 요약")
            
            with gr.Row():
                with gr.Column():
                    keywords_output = gr.Dataframe(
                        headers=["키워드", "빈도수"],
                        label="주요 키워드"
                    )
                with gr.Column():
                    chart_output = gr.Plot(label="키워드 빈도 차트")
        
        with gr.TabItem("상세 분석 결과"):
            articles_output = gr.HTML(label="기사별 분석 결과")
        
        with gr.TabItem("JSON 데이터"):
            json_output = gr.JSON(label="원시 데이터")
    
    analyze_btn.click(
        process_query,
        inputs=[query_input, num_articles],
        outputs=[summary_output, keywords_output, chart_output, articles_output, json_output]
    )
    
    gr.Markdown("## 사용 방법")
    gr.Markdown("""
    1. 분석할 주제를 입력합니다 (예: '인공지능', '기후변화', '경제 위기' 등).
    2. 분석할 뉴스 기사 수를 선택합니다.
    3. '뉴스 분석하기' 버튼을 클릭합니다.
    4. 분석 결과는 세 개의 탭에 나누어 표시됩니다:
        - 요약 및 키워드: 전체 분석 요약과 주요 키워드 및 차트
        - 상세 분석 결과: 각 기사별 분석 내용 (요약, 감성, 키워드)
        - JSON 데이터: 원시 분석 데이터
    """)
    

demo.launch(share=False)

In [ ]:
demo.close()

---

## **데이터 처리 함수 구현**


In [ ]:
import requests, os
from langchain_core.tools import tool
from typing import Dict, Literal
import datetime
import re
from typing import List, Dict, Any, Optional
from email.utils import parsedate_to_datetime
from dateutil import parser as date_parser
import pytz
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
import bs4

`(1) 네이버 뉴스 검색 `

In [ ]:
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다.

    Args:
        query (str): 검색어

    Returns:
        Dict[Dict, int]: 검색 결과와 상태 코드  
    """


    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {
        "query": query,
        "display": 20,
        }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None
    
    # JSON 응답을 파싱하여 데이터를 반환
    return response.json()["items"]  

news_items = naver_news_search("인공지능")

print(f"검색된 뉴스 기사 수: {len(news_items)}")
print(f"첫 번째 뉴스 기사 제목: {news_items[0]['title']}")

`(2) 뉴스 데이터 전처리`

In [ ]:
def filter_news_by_date_naver(
    news_data: List[Dict[str, Any]], 
    days_limit: int = 7
    ) -> List[Dict[str, Any]]:
    """
    네이버 뉴스 검색 API 결과를 최근 기간으로 필터링하는 함수
    
    Args:
        news_data (List[Dict]): 검색된 뉴스 데이터 리스트
        days_limit (int): 최근 몇 일 동안의 뉴스만 필터링할지 (기본값: 7일)
        
    Returns:
        List[Dict]: 최근 기간으로 필터링된 뉴스 데이터 리스트
    """
    if not news_data:
        return []
    
    filtered_news = []

    # timezone 정보가 없는(naive) 현재 날짜 사용
    current_date = datetime.datetime.now()
    cutoff_date = current_date - datetime.timedelta(days=days_limit)
    
    for i, news in enumerate(news_data):
        # 날짜 필드 찾기 (네이버 API는 'pubDate' 필드 사용)
        pub_date = news.get('pubDate')
        
        # 날짜 필드가 없는 경우 포함하지 않음
        if not pub_date:
            print(f"뉴스 {i+1}: 날짜 필드 없음")
            continue
            
        try:
            # 다양한 날짜 형식 처리
            news_date = parse_date_flexible(pub_date)
            
            if not news_date:
                print(f"뉴스 {i+1}: 날짜 형식 인식 불가 - {pub_date}")
                continue
            
            # timezone 정보가 있는(aware) 날짜를 naive로 변환
            if news_date.tzinfo is not None:
                news_date = news_date.replace(tzinfo=None)
                        
            # 설정한 기간 이내의 뉴스만 포함
            if news_date >= cutoff_date:
                filtered_news.append(news)
                
        except Exception as e:
            print(f"뉴스 {i+1}: 날짜 파싱 오류 - {str(e)}")
            continue
    
    return filtered_news

def parse_date_flexible(date_str: str) -> Optional[datetime.datetime]:
    """
    다양한 형식의 날짜 문자열을 파싱하는 함수
    
    Args:
        date_str (str): 파싱할 날짜 문자열
        
    Returns:
        Optional[datetime.datetime]: 파싱된 날짜, 파싱 실패 시 None
        
    Note:
        반환된 날짜는 timezone 정보가 포함될 수 있습니다 (offset-aware).
        비교하기 전에 timezone 정보를 제거해야 합니다.
    """
    # 빈 문자열 체크
    if not date_str or not isinstance(date_str, str):
        return None
        
    # 1. RFC 822 형식 시도 (네이버 API 공식 문서 형식)
    try:
        return parsedate_to_datetime(date_str)
    except (ValueError, TypeError):
        pass
    
    # 2. 일반적인 ISO 형식 시도
    try:
        return datetime.datetime.fromisoformat(date_str.replace('Z', '+00:00'))
    except (ValueError, TypeError):
        pass
    
    # 3. 네이버 뉴스에서 자주 사용하는 형식 처리
    # 예: '2023년 10월 15일 오후 2시 30분'
    korean_pattern = r'(\d{4})년\s*(\d{1,2})월\s*(\d{1,2})일\s*(오전|오후)?\s*(\d{1,2})시\s*(\d{1,2})분'
    match = re.search(korean_pattern, date_str)
    if match:
        year, month, day, ampm, hour, minute = match.groups()
        hour = int(hour)
        if ampm == '오후' and hour < 12:
            hour += 12
        elif ampm == '오전' and hour == 12:
            hour = 0
        
        try:
            return datetime.datetime(int(year), int(month), int(day), hour, int(minute))
        except (ValueError, TypeError):
            pass
    
    # 4. 다른 형식의 날짜 (YYYY-MM-DD, YYYY/MM/DD 등)
    # dateutil 라이브러리 사용 (pip install python-dateutil 필요)
    try:
        return date_parser.parse(date_str)
    except (ValueError, TypeError):
        pass
    
    # 5. 원본 문자열에서 일자만 추출 시도 (마지막 방법)
    # 예: "Mon, 18 Mar 2025 14:30:45 +0900" 같은 형식에서 날짜만 추출
    date_patterns = [
        r'(\d{1,2})\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+(\d{4})',  # 18 Mar 2025
        r'(\d{4})-(\d{1,2})-(\d{1,2})',  # 2025-03-18
        r'(\d{1,2})/(\d{1,2})/(\d{4})'   # 3/18/2025 or 18/3/2025
    ]
    
    for pattern in date_patterns:
        match = re.search(pattern, date_str)
        if match:
            try:
                # 재구성된 간단한 형식으로 다시 시도
                simplified = ' '.join(match.groups())
                return date_parser.parse(simplified)
            except (ValueError, TypeError):
                pass
    
    # 로그에 파싱 실패한 날짜 형식 출력 (디버깅용)
    print(f"파싱 실패한 날짜 형식: {date_str}")
    
    # 모든 시도 실패
    return None


# 최근 5일 내 뉴스만 필터링
filtered_news = filter_news_by_date_naver(news_items, days_limit=5)
print(f"필터링된 뉴스 기사 수: {len(filtered_news)}")

In [ ]:
# 불필요한 공백 문자 제거
def clean_text(text: str) -> str:
    """
    텍스트에서 불필요한 공백, 줄바꿈 문자를 제거하는 함수
    """

    cleaned_text = str(text)

    # 연속된 줄바꿈을 하나로 통합
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)

    # 연속된 공백을 하나로 통합
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    # 줄 시작과 끝의 공백 제거
    cleaned_text = re.sub(r'^\s+|\s+$', '', cleaned_text, flags=re.MULTILINE)

    return cleaned_text
    

# 추출할 정보를 정의하는 Pydantic 모델 생성
class NewsArticle(BaseModel):
    """뉴스 기사에서 추출할 정보."""
    
    media_outlet: Optional[str] = Field(
        default=None, 
        description="뉴스 기사를 발행한 언론사 이름"
    )
    reporter: Optional[str] = Field(
        default=None, 
        description="기사를 작성한 기자의 이름"
    )
    content: Optional[str] = Field(
        default=None, 
        description="뉴스 기사의 본문 내용"
    )

# 추출 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 뉴스 기사에서 정보를 추출하는 전문가입니다. "
            "주어진 텍스트에서 관련 정보만 추출하세요. "
            "추출할 수 없는 정보가 있다면 해당 필드의 값으로 null을 반환하세요."
        ),
        ("human", "{text}")
    ]
)

# 모델 및 구조화된 출력 설정
llm = ChatOpenAI(model="gpt-4.1-mini") 
structured_llm = llm.with_structured_output(NewsArticle)

# 추출 파이프라인 생성
extraction_chain = prompt_template | structured_llm

In [ ]:
# 뉴스 검색부터 본문 추출까지의 파이프라인 함수
def extract_articles(query=str, num_articles=int):
    """사용자 질문을 처리하고 결과를 반환하는 함수"""

    # 쿼리 전처리
    query = query.strip()

    # 뉴스 기사 검색
    news_items = naver_news_search(query)
    
    # 최근 기간으로 필터링
    filtered_news = filter_news_by_date_naver(news_items, days_limit=5)

    # 각 뉴스 기사에서 정보 추출
    processed_docs = []

    for news in filtered_news:
        url = news['link']
        title = news['title']

        # 네이버 뉴스 기사 링크에서만 추출
        if "naver.com" not in url:
            continue

        loader = WebBaseLoader(
            url, 
            header_template={"User-Agent": "Mozilla/5.0"},
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=("media_and_head", "newsct_article _article_body"),
                )
            ),
        )
        
        # 뉴스 본문 로드
        doc = loader.load()[0]

        # 불필요한 공백 문자 제거
        doc.page_content = clean_text(doc.page_content)
        
        # 정보 추출 수행
        result = extraction_chain.invoke({"text": doc.page_content})
        
        # 추출된 정보를 메타데이터에 추가
        doc.metadata["title"] = title
        doc.metadata.update(result.model_dump())

        # 처리된 문서 리스트에 추가
        processed_docs.append(doc)

        # 기사 수 제한
        if len(processed_docs) >= num_articles:
            break

    
    return processed_docs


# 테스트
query = "인공지능"
num_articles = 3
processed_docs = extract_articles(query, num_articles)
print(f"처리된 문서 수: {len(processed_docs)}")
print(f"첫 번째 문서 메타데이터: {processed_docs[0].metadata}")
print(f"첫 번째 문서 본문 내용: {processed_docs[0].page_content[:200]}...")  # 첫 200자만 출력

`(3) LLM 데이터 분석 체인`

In [ ]:
# 요약 프롬프트 템플릿 정의
summarization_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 텍스트를 간결하게 요약하세요. 핵심 내용만 포함하고 중요한 정보는 유지하세요. (공백 포함 100글자 이내로 요약)"),
    ("human", "{text}")
])

# 요약 체인 생성
summarization_chain = summarization_prompt | llm | StrOutputParser()

In [ ]:
# 키워드 추출을 위한 Pydantic 모델
class Keywords(BaseModel):
    """텍스트에서 추출된 키워드 목록"""
    keywords: List[str] = Field(description="텍스트에서 추출된 중요 키워드 목록 ")

# 키워드 추출 프롬프트 템플릿 정의
keyword_extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 텍스트에서 중요한 키워드를 추출하세요. 키워드는 텍스트의 핵심 주제와 개념을 나타내야 합니다. (최대 10개)"),
    ("human", "{text}")
])

# 구조화된 출력을 위한 LLM 설정
keyword_extraction_llm = llm.with_structured_output(Keywords)

# 키워드 추출 체인 생성
keyword_extraction_chain = keyword_extraction_prompt | keyword_extraction_llm

In [ ]:
# 감성 분석 결과를 위한 Pydantic 모델
class SentimentAnalysis(BaseModel):
    """뉴스 기사의 감성 분석 결과"""
    sentiment: Literal["긍정적", "부정적", "중립적"] = Field(description="감성 분석 결과 (긍정적, 부정적, 중립적)")
    score: float = Field(description="감성 점수 (0~1, 1에 가까울수록 긍정적)")
    key_phrases: List[str] = Field(description="기사에서 감성을 나타내는 주요 구문")
    explanation: str = Field(description="감성 분석 결과에 대한 설명")

# 구조화된 출력을 위한 LLM 설정
structured_llm = llm.with_structured_output(SentimentAnalysis)

# 감성 분석 프롬프트 템플릿
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 뉴스 기사의 감성을 분석하는 전문가입니다. 
    주어진 뉴스 기사를 분석하여 긍정적, 부정적, 중립적 감성을 파악하세요.
    감성 점수는 0(매우 부정적)에서 1(매우 긍정적) 사이의 값으로 제공하세요.
    기사에서 감성을 나타내는 주요 구문을 추출하고 분석 결과를 설명하세요."""),
    ("human", "{news_article}")
])

# 감성 분석 체인 생성
sentiment_analysis_chain = sentiment_prompt | structured_llm

`(4) process_query 함수 정의`

In [ ]:
def process_query(query, num_articles):
    """사용자 질문을 처리하고 결과를 반환하는 함수"""

    # 쿼리 전처리
    query = query.strip()
    
    if not query:
        return "주제를 입력해주세요.", [], None, "분석 결과가 없습니다.", None
    
    # 뉴스 검색
    articles = extract_articles(query, num_articles)
    
    if not articles:
        return "검색 결과가 없습니다.", [], None, "분석 결과가 없습니다.", None
    
    results = []
    all_text = ""
    
    # 각 기사 분석
    for article_info in articles:
        article = article_info.page_content
        if article:
            # 요약
            summary = summarization_chain.invoke({"text": article})
            # 키워드 추출
            keywords = keyword_extraction_chain.invoke({"text": article})
            # 감성 분석
            sentiment_analysis = sentiment_analysis_chain.invoke({"news_article": article})
            # 결과 저장
            result = {
                "title": article_info.metadata["title"],
                "url": article_info.metadata["source"],
                "summary": summary,
                "sentiment": sentiment_analysis.sentiment,
                "sentiment_score": sentiment_analysis.score,
                "keywords": keywords.keywords
            }
            results.append(result)

            # 전체 텍스트에 추가
            all_text += article + "\n\n"
    
    if not results:
        return "기사 분석에 실패했습니다.", [], None, "분석 결과가 없습니다.", None

    # 키워드 빈도 계산
    keyword_counts = {}
    for result in results:
        for keyword in result['keywords']:
            keyword_counts[keyword] = keyword_counts.get(keyword, 0) + 1

    # 키워드 정렬
    all_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)

    # 상위 10개 키워드
    top_keywords = all_keywords[:10]


    # 감성 분석 결과 종합
    sentiment_counts = {"긍정적": 0, "중립적": 0, "부정적": 0}
    for result in results:
        sentiment_counts[result['sentiment']] += 1

    # 한글 폰트 
    import matplotlib.pyplot as plt
    plt.rcParams['font.family'] = 'AppleGothic'
    plt.rcParams['axes.unicode_minus'] = False
    
    # 차트 출력
    def generate_keyword_chart(keywords):
        """키워드 빈도 차트 생성"""
        words = [k[0] for k in keywords]
        counts = [k[1] for k in keywords]
        
        plt.figure(figsize=(10, 6))
        plt.bar(words, counts, color='skyblue')
        plt.xticks(rotation=45, ha='right')
        plt.title('주요 키워드 빈도')
        plt.tight_layout()
        
        return plt   
    
    keyword_chart = generate_keyword_chart(all_keywords)
    
    # 결과 요약 생성
    summary_text = f"총 {len(results)}개의 뉴스 기사를 분석했습니다.\n\n"    
    summary_text += f"감성 분석 결과: 긍정적 {sentiment_counts['긍정적']}개, 중립적 {sentiment_counts['중립적']}개, 부정적 {sentiment_counts['부정적']}개\n\n"
    
    # 결과 목록 생성
    articles_html = ""
    for i, result in enumerate(results, 1):
        articles_html += f"<div style='margin-bottom: 20px; padding: 15px; border: 1px solid #ddd; border-radius: 5px;'>"
        articles_html += f"<h3>{i}. {result['title']}</h3>"
        articles_html += f"<p><strong>URL:</strong> <a href='{result['url']}' target='_blank'>{result['url']}</a></p>"
        articles_html += f"<p><strong>요약:</strong> {result['summary']}</p>"
        articles_html += f"<p><strong>감성:</strong> {result['sentiment']} (점수: {result['sentiment_score']:.2f})</p>"
        
        # 키워드 태그 스타일로 표시
        articles_html += f"<p><strong>주요 키워드:</strong> "
        for idx, (keyword, count) in enumerate(top_keywords):
            # 여러 색상으로 순환
            colors = ['#4299e1', '#48bb78', '#ed8936', '#9f7aea', '#ed64a6']
            bg_color = colors[idx % len(colors)]
            articles_html += f"<span style='display: inline-block; background-color: {bg_color}; color: white; padding: 3px 8px; margin: 2px; border-radius: 10px; font-weight: 500;'>{keyword} ({count})</span>"
        articles_html += "</p></div>"
    
    return summary_text, all_keywords, keyword_chart, articles_html, results

In [ ]:
# 테스트
summary_text, all_keywords, keyword_chart, articles_html, results = process_query("딥시크 한국 진출", 3)

print(summary_text)
print(f"주요 키워드: {all_keywords}")

In [ ]:
print(articles_html)

In [ ]:
results

---

### **Gradio 실행**

In [ ]:
import gradio as gr

# Gradio 인터페이스 구현
with gr.Blocks(title="뉴스 분석 대시보드", theme=gr.themes.Soft(), analytics_enabled=False) as demo:
    gr.Markdown("# 🔍 뉴스 분석 대시보드")
    gr.Markdown("주제를 입력하면 관련 뉴스를 검색하고 요약, 키워드 추출, 감성 분석을 수행합니다.")
    
    with gr.Row():
        with gr.Column(scale=4):
            query_input = gr.Textbox(label="분석할 주제 입력", placeholder="예: 인공지능, 기후변화, 경제 등")
        with gr.Column(scale=1):
            num_articles = gr.Slider(label="분석할 기사 수", minimum=1, maximum=10, value=3, step=1)
    
    analyze_btn = gr.Button("뉴스 분석하기", variant="primary")
    
    # 결과 출력 영역
    with gr.Tabs():
        with gr.TabItem("요약 및 키워드"):
            summary_output = gr.Textbox(label="분석 요약")
            
            with gr.Row():
                with gr.Column():
                    keywords_output = gr.Dataframe(
                        headers=["키워드", "빈도수"],
                        label="주요 키워드"
                    )
                with gr.Column():
                    chart_output = gr.Plot(label="키워드 빈도 차트")
        
        with gr.TabItem("상세 분석 결과"):
            articles_output = gr.HTML(label="기사별 분석 결과")
        
        with gr.TabItem("JSON 데이터"):
            json_output = gr.JSON(label="원시 데이터")
    
    analyze_btn.click(
        process_query,
        inputs=[query_input, num_articles],
        outputs=[summary_output, keywords_output, chart_output, articles_output, json_output]
    )
    
    gr.Markdown("## 사용 방법")
    gr.Markdown("""
    1. 분석할 주제를 입력합니다 (예: '인공지능', '기후변화', '경제 위기' 등).
    2. 분석할 뉴스 기사 수를 선택합니다.
    3. '뉴스 분석하기' 버튼을 클릭합니다.
    4. 분석 결과는 세 개의 탭에 나누어 표시됩니다:
        - 요약 및 키워드: 전체 분석 요약과 주요 키워드 및 차트
        - 상세 분석 결과: 각 기사별 분석 내용 (요약, 감성, 키워드)
        - JSON 데이터: 원시 분석 데이터
    """)
    

demo.launch(share=False)

In [ ]:
demo.close()

---

### **[심화] Gradio 인터페이스 구현 및 Hugging Face Spaces 배포** 

#### 1. **사전 준비**
- [Hugging Face](https://huggingface.co/spaces) 계정 생성이 필요 (무료)
- 로컬에서 작동하는 Gradio 앱을 준비
- 새로운 **가상 환경**에서 실행 (crwal4ai 의존성과 gradio 최신 버전 문제)
- **app.py** 파일을 생성하여 프로젝트 폴더로 복사 (.env 파일도 함께 복사)

- pyproject.toml

- 터미널에서 다음 명령어 실행

- 서버 종료: 터미널에서 Ctrl + C

- requirements.txt 파일 생성 

- .gitignore 파일 생성

#### 2. **배포 방법 (터미널 이용)**
- Gradio 앱이 있는 디렉토리로 이동 
- 터미널에서 다음 명령어 실행


- CLI가 안내하는 대로 기본 메타데이터 입력
   - HF Token : Hugging Face 계정의 Access Token 복사해서 붙여넣고 엔터 
   - git credential 설정 (y/n) : n
   - Space 이름
   - Any Spaces secrets (y/n) : env 환경변수 설정 (OPENAI_API_KEY 붙여 넣고 엔터)
   - 또는 HF Spaces Secrets에 직접 추가 (OPENAI_API_KEY 값 입력 후 엔터)
- 배포 완료 후 제공되는 URL로 접근 가능

---

### **[실습 프로젝트]**

- 네이버 뉴스 URL을 입력하면, 뉴스 본문을 자동 요약하고 뉴스 감성 분석을 수행하는 과정을 코드로 구현합니다. 
- Gradio 인터페이스에 적용하여 구현합니다. 

In [ ]:
# 여기에 코드를 작성하세요. 